In [20]:
from pathlib import Path
from datetime import datetime
import os, time, json, shutil, subprocess

# ============================================================
# CONFIG — RUN NOTEBOOK FROM MomentumSystem ROOT OR SET PATH
# ============================================================

# Option A (recommended): run notebook from MomentumSystem root
#PROJECT_ROOT = Path.cwd()

# Option B: hardcode it (uncomment and set)
PROJECT_ROOT = Path(r"C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem")

SOURCE_FOLDERS = [
    "13-match_trade_generator_output_regression_insp500_spyfilter_cap15",
    "13-trading_output_regression_insp500_spyfilter_cap15",
    "27a-2G_live_trading",
]

# Your new folders (you said you created them)
PUBLISH_ROOT    = PROJECT_ROOT / "_publish"
LOG_DIR         = PROJECT_ROOT / "_logs"
SYNC_DIR        = PROJECT_ROOT / "_sync"
MANIFEST_DIR    = PROJECT_ROOT / "_manifest"
QUARANTINE_DIR  = PROJECT_ROOT / "_quarantine"
SNAPSHOT_DIR    = PROJECT_ROOT / "_snapshots"   # optional; not required

# rclone executable (set to "rclone" if in PATH)
RCLONE_EXE = "rclone"  # or r"C:\Tools\rclone\rclone.exe"

# Dropbox remote name (configured via `rclone config`)
DROPBOX_REMOTE = "dropbox:MomentumSystem"  # you can change the folder name in Dropbox

REMOTE_LATEST  = f"{DROPBOX_REMOTE}/latest"
REMOTE_CHANGES = f"{DROPBOX_REMOTE}/_changes"

# What to sync
INCLUDE_EXTS = {".parquet", ".csv", ".json", ".log", ".txt"}

# Exclude temp/lock
EXCLUDE_SUFFIXES = {".tmp", ".lock"}

# Stability gate: file must be unchanged for N seconds to be "safe"
STABLE_SECONDS = 10

# Optional verification (can be slow)
DO_RCLONE_CHECK = False

# Optional backup retention in Dropbox changes folder (days). Set None to disable.
CHANGES_RETENTION_DAYS = 60

print("PROJECT_ROOT =", PROJECT_ROOT)


PROJECT_ROOT = C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem


In [21]:
def ensure_dirs():
    for p in [PUBLISH_ROOT, LOG_DIR, SYNC_DIR, MANIFEST_DIR, QUARANTINE_DIR]:
        p.mkdir(parents=True, exist_ok=True)
    # snapshots optional
    SNAPSHOT_DIR.mkdir(parents=True, exist_ok=True)

def run_cmd(cmd):
    print(">>>", " ".join(cmd))
    p = subprocess.run(cmd, capture_output=True, text=True)
    if p.stdout:
        print(p.stdout)
    if p.returncode != 0:
        if p.stderr:
            print(p.stderr)
        raise RuntimeError(f"Command failed (exit {p.returncode}): {' '.join(cmd)}")
    return p.stdout

def ensure_rclone():
    # sanity check: rclone present
    run_cmd([RCLONE_EXE, "version"])

def ensure_remote_dirs():
    run_cmd([RCLONE_EXE, "mkdir", DROPBOX_REMOTE])   # dropbox:MomentumSystem
    run_cmd([RCLONE_EXE, "mkdir", REMOTE_LATEST])    # dropbox:MomentumSystem/latest
    run_cmd([RCLONE_EXE, "mkdir", REMOTE_CHANGES])   # dropbox:MomentumSystem/_changes
  
import os, shutil, subprocess

def refresh_windows_path_into_python():
    """Pull Machine+User PATH from Windows and set it in this Python process."""
    cmd = [
        "powershell", "-NoProfile", "-Command",
        "[System.Environment]::GetEnvironmentVariable('Path','Machine') + ';' + " +
        "[System.Environment]::GetEnvironmentVariable('Path','User')"
    ]
    new_path = subprocess.check_output(cmd, text=True).strip()
    # prepend so it's definitely used
    os.environ["PATH"] = new_path + ";" + os.environ.get("PATH", "")

def resolve_rclone_exe():
    """Return an executable name/path that subprocess can run."""
    # 1) try plain 'rclone' in current PATH
    p = shutil.which("rclone")
    if p:
        return p

    # 2) refresh PATH from Windows registry, try again
    refresh_windows_path_into_python()
    p = shutil.which("rclone")
    if p:
        return p

    # 3) last resort: ask PowerShell where rclone is and use the absolute path
    cmd = [
        "powershell", "-NoProfile", "-Command",
        "(Get-Command rclone -ErrorAction Stop).Source"
    ]
    try:
        p = subprocess.check_output(cmd, text=True).strip()
        if p and os.path.exists(p):
            return p
    except subprocess.CalledProcessError:
        pass

    raise FileNotFoundError(
        "Jupyter still cannot locate rclone. Close/reopen Jupyter (or restart the kernel) "
        "after installing rclone, or set RCLONE_EXE to the full path of rclone.exe."
    )

# IMPORTANT: overwrite your RCLONE_EXE setting here
RCLONE_EXE = resolve_rclone_exe()
print("Using RCLONE_EXE =", RCLONE_EXE)


def write_rclone_filter():
    """
    Keep filters in _sync so it's operational/config stuff.
    """
    filter_path = SYNC_DIR / "rclone_filters.txt"
    rules = []
    # include desired extensions
    for ext in sorted(INCLUDE_EXTS):
        rules.append(f"+ *{ext}")
    # exclude temp/locks
    for suf in sorted(EXCLUDE_SUFFIXES):
        rules.append(f"- *{suf}")
    # exclude junk
    rules += [
        "- **/.git/**",
        "- **/__pycache__/**",
        "- **/~*",
        "- **/*.bak",
    ]
    # exclude everything else
    rules.append("- *")

    filter_path.write_text("\n".join(rules) + "\n", encoding="utf-8")
    return filter_path

LOCK_FILE = SYNC_DIR / "sync_lock.json"

def acquire_lock(max_age_minutes=30):
    """Acquire lock, but auto-clear if stale (older than max_age_minutes)."""
    if LOCK_FILE.exists():
        try:
            lock_data = json.loads(LOCK_FILE.read_text(encoding="utf-8"))
            created = datetime.fromisoformat(lock_data["created"])
            age_minutes = (datetime.now() - created).total_seconds() / 60
            
            if age_minutes > max_age_minutes:
                print(f"⚠ Stale lock detected ({age_minutes:.1f} min old, PID {lock_data.get('pid')}). Auto-removing.")
                LOCK_FILE.unlink()
            else:
                raise RuntimeError(
                    f"Lock exists: {LOCK_FILE}. "
                    f"Created {age_minutes:.1f} minutes ago (PID {lock_data.get('pid')}). "
                    "Another run may be in progress. "
                    f"Wait {max_age_minutes - age_minutes:.1f} more minutes or manually delete if crashed."
                )
        except (json.JSONDecodeError, KeyError, ValueError) as e:
            print(f"⚠ Corrupted lock file. Removing it. Error: {e}")
            LOCK_FILE.unlink()
    
    LOCK_FILE.write_text(json.dumps({
        "created": datetime.now().isoformat(timespec="seconds"),
        "pid": os.getpid(),
    }, indent=2), encoding="utf-8")
    print(f"✓ Lock acquired (PID {os.getpid()})")

def release_lock():
    if LOCK_FILE.exists():
        LOCK_FILE.unlink()

ensure_dirs()
ensure_rclone()
FILTER_FILE = write_rclone_filter()
ensure_remote_dirs()
print("Filter file:", FILTER_FILE)



Using RCLONE_EXE = C:\Users\farty\AppData\Local\Microsoft\WinGet\Links\rclone.EXE
>>> C:\Users\farty\AppData\Local\Microsoft\WinGet\Links\rclone.EXE version
rclone v1.72.1
- os/version: Microsoft Windows 11 Home 24H2 24H2 (64 bit)
- os/kernel: 10.0.26100.7462 (x86_64)
- os/type: windows
- os/arch: amd64
- go/version: go1.25.5
- go/linking: static
- go/tags: cmount

>>> C:\Users\farty\AppData\Local\Microsoft\WinGet\Links\rclone.EXE mkdir dropbox:MomentumSystem
>>> C:\Users\farty\AppData\Local\Microsoft\WinGet\Links\rclone.EXE mkdir dropbox:MomentumSystem/latest
>>> C:\Users\farty\AppData\Local\Microsoft\WinGet\Links\rclone.EXE mkdir dropbox:MomentumSystem/_changes
Filter file: C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_sync\rclone_filters.txt


In [22]:
def is_candidate_file(path: Path) -> bool:
    if not path.is_file():
        return False
    if path.suffix.lower() not in INCLUDE_EXTS:
        return False
    lname = path.name.lower()
    return not any(lname.endswith(suf) for suf in EXCLUDE_SUFFIXES)

def file_signature(path: Path):
    st = path.stat()
    return (st.st_size, st.st_mtime)

def wait_until_stable(path: Path, stable_seconds=STABLE_SECONDS, poll=1.0) -> bool:
    if not path.exists():
        return False
    sig0 = file_signature(path)
    t0 = time.time()
    while True:
        time.sleep(poll)
        if not path.exists():
            return False
        sig1 = file_signature(path)
        if sig1 != sig0:
            sig0 = sig1
            t0 = time.time()
        if (time.time() - t0) >= stable_seconds:
            return True

def needs_copy(src: Path, dst: Path) -> bool:
    if not dst.exists():
        return True
    return file_signature(src) != file_signature(dst)

def publish_one_folder(src_root: Path, dst_root: Path):
    copied = skipped = unstable = 0
    for root, _, files in os.walk(src_root):
        rootp = Path(root)
        for f in files:
            src = rootp / f
            if not is_candidate_file(src):
                continue
            ok = wait_until_stable(src)
            rel = src.relative_to(src_root)
            if not ok:
                # quarantine a reference copy (optional)
                qdst = QUARANTINE_DIR / src_root.name / rel
                qdst.parent.mkdir(parents=True, exist_ok=True)
                try:
                    shutil.copy2(src, qdst)
                except Exception:
                    pass
                unstable += 1
                continue
            
            # IMPORTANT: Normalize path separators to forward slashes for cross-platform compatibility
            # Convert Windows backslashes to forward slashes so rclone treats paths consistently
            rel_posix = Path(str(rel).replace('\\', '/'))
            dst = dst_root / rel_posix
            dst.parent.mkdir(parents=True, exist_ok=True)
            
            if needs_copy(src, dst):
                shutil.copy2(src, dst)
                copied += 1
            else:
                skipped += 1
    return {"copied": copied, "skipped": skipped, "unstable": unstable}

def publish_all():
    summary = {}
    for folder in SOURCE_FOLDERS:
        src = PROJECT_ROOT / folder
        dst = PUBLISH_ROOT / folder

        if not src.exists():
            raise FileNotFoundError(f"Missing source folder: {src}")

        dst.mkdir(parents=True, exist_ok=True)
        print(f"\nPublishing {src} -> {dst}")
        summary[folder] = publish_one_folder(src, dst)
        print("  ", summary[folder])

    ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    manifest = {
        "timestamp": ts,
        "project_root": str(PROJECT_ROOT),
        "published": summary
    }

    (MANIFEST_DIR / f"manifest_{ts}.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")
    (PUBLISH_ROOT / "RUN_COMPLETE.txt").write_text(ts + "\n", encoding="utf-8")
    return manifest

manifest = publish_all()
manifest



Publishing C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\13-match_trade_generator_output_regression_insp500_spyfilter_cap15 -> C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_publish\13-match_trade_generator_output_regression_insp500_spyfilter_cap15
   {'copied': 5, 'skipped': 0, 'unstable': 0}

Publishing C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\13-trading_output_regression_insp500_spyfilter_cap15 -> C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_publish\13-trading_output_regression_insp500_spyfilter_cap15
   {'copied': 4, 'skipped': 0, 'unstable': 0}

Publishing C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\27a-2G_live_trading -> C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_publish\27a-2G_live_trading
   {'copied': 8, 'skipped': 0, 'unstable': 0}


{'timestamp': '2026-01-09_105908',
 'project_root': 'C:\\TWS API\\source\\pythonclient\\TradingIdeas\\MomentumSystem',
 'published': {'13-match_trade_generator_output_regression_insp500_spyfilter_cap15': {'copied': 5,
   'skipped': 0,
   'unstable': 0},
  '13-trading_output_regression_insp500_spyfilter_cap15': {'copied': 4,
   'skipped': 0,
   'unstable': 0},
  '27a-2G_live_trading': {'copied': 8, 'skipped': 0, 'unstable': 0}}}

In [23]:
def rclone_sync(local_dir: Path, remote_dir: str, backup_dir: str, log_path: Path):
    cmd = [
        RCLONE_EXE, "sync",
        str(local_dir),
        remote_dir,
        "--filter-from", str(FILTER_FILE),
        "--backup-dir", backup_dir,
        "--retries", "5",
        "--low-level-retries", "20",
        "--log-level", "INFO",
        "--log-file", str(log_path),
    ]
    run_cmd(cmd)

def rclone_check(local_dir: Path, remote_dir: str, log_path: Path):
    cmd = [
        RCLONE_EXE, "check",
        str(local_dir),
        remote_dir,
        "--filter-from", str(FILTER_FILE),
        "--log-level", "INFO",
        "--log-file", str(log_path),
    ]
    run_cmd(cmd)

def sync_all():
    marker = PUBLISH_ROOT / "RUN_COMPLETE.txt"
    if not marker.exists():
        raise RuntimeError(f"RUN_COMPLETE.txt not found: {marker}. Publish step may not have completed.")

    run_ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")

    for folder in SOURCE_FOLDERS:
        local_dir  = PUBLISH_ROOT / folder
        remote_dir = f"{REMOTE_LATEST}/{folder}"
        backup_dir = f"{REMOTE_CHANGES}/{run_ts}/{folder}"

        log_sync = LOG_DIR / f"rclone_sync_{folder}_{run_ts}.log"
        print(f"\nSYNC {local_dir} -> {remote_dir}")
        rclone_sync(local_dir, remote_dir, backup_dir, log_sync)

        if DO_RCLONE_CHECK:
            log_check = LOG_DIR / f"rclone_check_{folder}_{run_ts}.log"
            print(f"CHECK {local_dir} <-> {remote_dir}")
            rclone_check(local_dir, remote_dir, log_check)

acquire_lock()
try:
    sync_all()
finally:
    release_lock()

print("Sync complete.")


✓ Lock acquired (PID 30688)

SYNC C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_publish\13-match_trade_generator_output_regression_insp500_spyfilter_cap15 -> dropbox:MomentumSystem/latest/13-match_trade_generator_output_regression_insp500_spyfilter_cap15
>>> C:\Users\farty\AppData\Local\Microsoft\WinGet\Links\rclone.EXE sync C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_publish\13-match_trade_generator_output_regression_insp500_spyfilter_cap15 dropbox:MomentumSystem/latest/13-match_trade_generator_output_regression_insp500_spyfilter_cap15 --filter-from C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_sync\rclone_filters.txt --backup-dir dropbox:MomentumSystem/_changes/2026-01-09_105908/13-match_trade_generator_output_regression_insp500_spyfilter_cap15 --retries 5 --low-level-retries 20 --log-level INFO --log-file C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_logs\rclone_sync_13-match_trade_generator_output_regression_insp500_spyfilte

In [24]:
def cleanup_old_changes(days: int):
    if not days:
        print("Retention cleanup disabled.")
        return

    # If folder doesn't exist yet, skip cleanly
    p = subprocess.run([RCLONE_EXE, "lsf", REMOTE_CHANGES], capture_output=True, text=True)
    if p.returncode != 0:
        print(f"Skip cleanup: remote path not found yet: {REMOTE_CHANGES}")
        return

    run_ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    log_del = LOG_DIR / f"rclone_delete_changes_{run_ts}.log"

    cmd = [
        RCLONE_EXE, "delete",
        REMOTE_CHANGES,
        "--min-age", f"{days}d",
        "--log-level", "INFO",
        "--log-file", str(log_del),
    ]
    run_cmd(cmd)

cleanup_old_changes(CHANGES_RETENTION_DAYS)


>>> C:\Users\farty\AppData\Local\Microsoft\WinGet\Links\rclone.EXE delete dropbox:MomentumSystem/_changes --min-age 60d --log-level INFO --log-file C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_logs\rclone_delete_changes_2026-01-09_105922.log


In [29]:
from pathlib import Path
print(Path(r"C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\_logs\rclone_delete_changes_2026-01-02_125631.log").read_text(errors="ignore"))


2026/01/02 12:56:31 ERROR : error listing: directory not found
2026/01/02 12:56:31 ERROR : Attempt 1/3 failed with 2 errors and: directory not found
2026/01/02 12:56:31 ERROR : error listing: directory not found
2026/01/02 12:56:31 ERROR : Attempt 2/3 failed with 2 errors and: directory not found
2026/01/02 12:56:32 ERROR : error listing: directory not found
2026/01/02 12:56:32 ERROR : Attempt 3/3 failed with 2 errors and: directory not found
2026/01/02 12:56:32 INFO  : Dropbox root 'MomentumSystem/_changes': Committing uploads - please wait...
2026/01/02 12:56:32 NOTICE: Failed to delete with 2 errors: last error was: directory not found



In [26]:
# Check Dropbox contents
import subprocess

# Compare file timestamps with dates
def compare_with_dates():
    for folder in SOURCE_FOLDERS:
        print(f"\n{'='*80}")
        print(f"FOLDER: {folder}")
        print('='*80)
        
        local_dir = PUBLISH_ROOT / folder
        
        # List local files with timestamps
        print("\nLOCAL FILES (_publish):")
        local_files = {}
        for f in sorted(local_dir.rglob("*")):
            if f.is_file() and is_candidate_file(f):
                mtime = datetime.fromtimestamp(f.stat().st_mtime)
                size = f.stat().st_size
                rel = f.relative_to(local_dir)
                local_files[str(rel)] = mtime
                print(f"  {mtime.strftime('%Y-%m-%d %H:%M:%S')}  {size:>10,} bytes  {rel}")
        
        # List Dropbox files with timestamps (rclone lsl format)
        print("\nDROPBOX FILES:")
        remote_path = f"{REMOTE_LATEST}/{folder}"
        cmd = [RCLONE_EXE, "lsl", remote_path]
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        dropbox_files = {}
        if result.returncode == 0 and result.stdout.strip():
            for line in result.stdout.strip().split('\n'):
                if line.strip():
                    # rclone lsl format: "    SIZE YYYY-MM-DD HH:MM:SS.mmm FILENAME"
                    parts = line.strip().split()
                    if len(parts) >= 4:
                        size = parts[0]
                        date = parts[1]
                        time = parts[2]
                        filename = ' '.join(parts[3:])
                        dropbox_files[filename] = f"{date} {time}"
                        print(f"  {date} {time}  {size:>10} bytes  {filename}")
        else:
            print(f"  (empty or error)")
        
        # Compare
        print("\nCOMPARISON:")
        all_files = set(local_files.keys()) | set(dropbox_files.keys())
        for fname in sorted(all_files):
            local_time = local_files.get(fname, "MISSING")
            dropbox_time = dropbox_files.get(fname, "MISSING")
            
            if local_time == "MISSING":
                print(f"  ⚠️  {fname}: Only in Dropbox")
            elif dropbox_time == "MISSING":
                print(f"  ⚠️  {fname}: Only in local (NOT SYNCED!)")
            else:
                # Compare
                local_dt = local_time
                dropbox_dt = datetime.strptime(dropbox_time.split('.')[0], '%Y-%m-%d %H:%M:%S')
                
                diff_seconds = (local_dt - dropbox_dt).total_seconds()
                if abs(diff_seconds) < 2:
                    status = "✓ SAME"
                elif diff_seconds > 0:
                    status = f"⚠️ LOCAL NEWER by {diff_seconds:.0f}s"
                else:
                    status = f"⚠️ DROPBOX NEWER by {-diff_seconds:.0f}s"
                
                print(f"  {status}: {fname}")
                print(f"      Local:   {local_time.strftime('%Y-%m-%d %H:%M:%S')}")
                print(f"      Dropbox: {dropbox_time}")

compare_with_dates()


FOLDER: 13-match_trade_generator_output_regression_insp500_spyfilter_cap15

LOCAL FILES (_publish):
  2026-01-08 19:05:35         492 bytes  13-all_planned_trades.csv
  2026-01-08 19:05:35       3,039 bytes  13-match_equity_curve_regression_insp500_spyfilter_cap15.parquet
  2026-01-08 19:05:35      10,790 bytes  13-match_trades_regression_insp500_spyfilter_cap15.parquet
  2026-01-08 19:05:35      12,525 bytes  13-match_weekly_rankings_pre_filter_cap15.parquet
  2026-01-08 19:05:35         492 bytes  planned_trades_LATEST.csv

DROPBOX FILES:
  2026-01-08 19:05:36.000000000         492 bytes  13-all_planned_trades.csv
  2026-01-08 19:05:36.000000000        3039 bytes  13-match_equity_curve_regression_insp500_spyfilter_cap15.parquet
  2026-01-08 19:05:36.000000000       10790 bytes  13-match_trades_regression_insp500_spyfilter_cap15.parquet
  2026-01-08 19:05:36.000000000       12525 bytes  13-match_weekly_rankings_pre_filter_cap15.parquet
  2026-01-08 19:05:36.000000000         492 byte

In [27]:
# Read the most recent sync logs
import glob

log_files = sorted(glob.glob(str(LOG_DIR / "rclone_sync_*.log")), reverse=True)[:3]

for log_file in log_files:
    print(f"\n{'='*80}")
    print(f"LOG: {Path(log_file).name}")
    print('='*80)
    with open(log_file, 'r', encoding='utf-8') as f:
        content = f.read()
        # Show just the summary or key lines
        if content.strip():
            print(content)
        else:
            print("(empty log)")


LOG: rclone_sync_27a-2G_live_trading_2026-01-09_105908.log
2026/01/09 10:59:15 INFO  : weekly_rankings/weekly_rankings_signal_20251217.csv: Moved (server-side)
2026/01/09 10:59:15 INFO  : weekly_rankings/weekly_rankings_signal_20251217.csv: Moved into backup dir
2026/01/09 10:59:16 INFO  : weekly_trades/weekly_trades_signal_20251231.csv: Moved (server-side)
2026/01/09 10:59:16 INFO  : weekly_trades/weekly_trades_signal_20251231.csv: Moved into backup dir
2026/01/09 10:59:18 INFO  : weekly_rankings/weekly_rankings_signal_20251224.csv: Moved (server-side)
2026/01/09 10:59:18 INFO  : weekly_rankings/weekly_rankings_signal_20251224.csv: Moved into backup dir
2026/01/09 10:59:19 INFO  : weekly_rankings/weekly_rankings_signal_20251231.csv: Moved (server-side)
2026/01/09 10:59:19 INFO  : weekly_rankings/weekly_rankings_signal_20251231.csv: Moved into backup dir
2026/01/09 10:59:20 INFO  : weekly_trades/weekly_trades_signal_20251224.csv: Moved (server-side)
2026/01/09 10:59:20 INFO  : weekly_

In [28]:
# Compare file timestamps: local _publish vs Dropbox
def compare_timestamps():
    for folder in SOURCE_FOLDERS:
        print(f"\n{'='*80}")
        print(f"FOLDER: {folder}")
        print('='*80)
        
        local_dir = PUBLISH_ROOT / folder
        
        # List local files with timestamps
        print("\nLOCAL FILES:")
        for f in sorted(local_dir.rglob("*")):
            if f.is_file() and is_candidate_file(f):
                mtime = datetime.fromtimestamp(f.stat().st_mtime)
                rel = f.relative_to(local_dir)
                print(f"  {mtime.strftime('%Y-%m-%d %H:%M:%S')} - {rel}")
        
        # List Dropbox files with timestamps
        print("\nDROPBOX FILES:")
        remote_path = f"{REMOTE_LATEST}/{folder}"
        cmd = [RCLONE_EXE, "lsl", remote_path]
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            for line in result.stdout.strip().split('\n'):
                if line.strip():
                    print(f"  {line}")
        else:
            print(f"  Error: {result.stderr}")

compare_timestamps()


FOLDER: 13-match_trade_generator_output_regression_insp500_spyfilter_cap15

LOCAL FILES:
  2026-01-08 19:05:35 - 13-all_planned_trades.csv
  2026-01-08 19:05:35 - 13-match_equity_curve_regression_insp500_spyfilter_cap15.parquet
  2026-01-08 19:05:35 - 13-match_trades_regression_insp500_spyfilter_cap15.parquet
  2026-01-08 19:05:35 - 13-match_weekly_rankings_pre_filter_cap15.parquet
  2026-01-08 19:05:35 - planned_trades_LATEST.csv

DROPBOX FILES:
  492 2026-01-08 19:05:36.000000000 13-all_planned_trades.csv
       3039 2026-01-08 19:05:36.000000000 13-match_equity_curve_regression_insp500_spyfilter_cap15.parquet
      10790 2026-01-08 19:05:36.000000000 13-match_trades_regression_insp500_spyfilter_cap15.parquet
      12525 2026-01-08 19:05:36.000000000 13-match_weekly_rankings_pre_filter_cap15.parquet
        492 2026-01-08 19:05:36.000000000 planned_trades_LATEST.csv

FOLDER: 13-trading_output_regression_insp500_spyfilter_cap15

LOCAL FILES:
  2026-01-08 19:05:59 - 13-equity_curve_re